# Neo4j RAG Pipeline with LangChain

In [5]:
pip install wikipedia

  Using cached wikipedia-1.4.0-py3-none-any.whl

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install --quiet langchain langchain-community langchain-neo4j langchain-openai langchain-experimental 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_DATABASE"] = os.getenv("NEO4J_DATABASE")
os.environ["AURA_INSTANCEID"] = os.getenv("AURA_INSTANCEID")
os.environ["AURA_INSTANCENAME"] = os.getenv("AURA_INSTANCENAME")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain.graphs import Neo4jGraph


graph = Neo4jGraph()

/var/folders/vp/07xvq8651z74827k4_nj65n80000gp/T/ipykernel_11662/799232279.py:4: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [6]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()
raw_documents

/Users/sreekanthpogula/.pyenv/versions/3.11.9/lib/python3.11/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/sreekanthpogula/.pyenv/versions/3.11.9/lib/python3.11/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [7]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents[:3])

In [18]:
documents

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [19]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_graph_transformer = LLMGraphTransformer(llm=llm)

In [20]:
graph_documents = llm_graph_transformer.convert_to_graph_documents(documents)

TypeError: unhashable type: 'dict'

In [22]:
graph_documents

NameError: name 'graph_documents' is not defined

In [21]:
graph.add_graph_documents(graph_documents,
                          baseEntityLabel=True,
                          include_source=True)

NameError: name 'graph_documents' is not defined

In [23]:
default_cipher = "MATCH (s)-[r:!Mentions]->(e) RETURN s, r, e LIMIT 50"

In [28]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
def show_graph(cypher: str = default_cipher):
    driver = GraphDatabase.driver(
        uri = os.getenv("NEO4J_URI"),
        auth = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    display(widget)
    return widget


ModuleNotFoundError: No module named 'yfiles_jupyter_graphs'

In [27]:
show_graph()

NameError: name 'GraphDatabase' is not defined